In [50]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [51]:
data=pd.read_csv("F:\\Epilepsy Data.csv")
data.drop('Unnamed: 0',axis=1,inplace=True)
data['y'][data['y']!=1]=0
cat=data['y']
cat=cat.as_matrix()
features=data.drop('y',axis=1)
features=features.as_matrix()
data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,0
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,1
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,0
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,0
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,0


In [52]:
test=data[:][data['y']==1]
test.drop('y',axis=1,inplace=True)

In [53]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [54]:
points=data.drop('y',axis=1).values
y=data['y'].values
model = KNeighborsClassifier(n_neighbors=3)
model.fit(points,y)
'''neigh = NearestNeighbors(n_neighbors=3)
neigh.fit(points) '''
ynew=[]

In [55]:
for xx in range(0,len(points)):
    ynew.append(model.predict([points[xx]])[0])

In [56]:
diff=data['y'].values-ynew

In [57]:
minorsafe=[]
minornoise=[]
majorsafe=[]
majornoise=[]
for x in range(0,len(ynew)):
    if ynew[x]!=y[x]:
        if y[x]==1:
            minornoise.append(points[x].tolist())
        else:
            majornoise.append(points[x].tolist())
    else:
        if y[x]==1:
            minorsafe.append(points[x].tolist())
        else:
            majorsafe.append(points[x].tolist())
print(len(minornoise))
print(len(majornoise))
print(len(minorsafe))
print(len(majorsafe))

450
1
1850
9199


In [58]:
temp=(data[:][data['y']==1])
temp.drop('y',axis=1,inplace=True)
minor=temp.values.tolist()
temp=(data[:][data['y']==0])
temp.drop('y',axis=1,inplace=True)
major=temp.values.tolist()

In [59]:
neigh = NearestNeighbors(n_neighbors=3)
neigh.fit(major)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=3, p=2, radius=1.0)

In [12]:
#WEAK-AMPLIFICATION
for x in range(0,len(minornoise)):
    temp=neigh.kneighbors([minornoise[x]])
    for xx in range(3):
        if major[temp[1][0][xx]] in majorsafe:
            minor.append(minornoise[x])

In [13]:
print(len(major))
print(len(minor))

9200
3649


In [14]:
y1=np.ones(len(minor))
y2=np.zeros(len(major))

In [15]:
yfinal=np.concatenate((y2,y1))

In [16]:
xfinal=np.concatenate((major,minor))

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( xfinal, yfinal, test_size=0.3, random_state=42)

In [18]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski' , p=2)
classifier.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [19]:
y_pred = classifier.predict(X_test)

In [20]:
classifier.score(X_test,y_test)

0.96160830090791183

In [21]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.95      1.00      0.97      2770
        1.0       0.99      0.87      0.93      1085

avg / total       0.96      0.96      0.96      3855



In [22]:
from sklearn.neural_network import MLPClassifier

In [23]:
model = MLPClassifier(hidden_layer_sizes=(100,100), max_iter=1000, batch_size=4,random_state=101)

In [24]:
model.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size=4, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=101,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [25]:
y_pred=model.predict(X_test)

In [26]:
model.score(X_test,y_test)

0.96238651102464334

In [27]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.97      0.98      0.97      2770
        1.0       0.95      0.92      0.93      1085

avg / total       0.96      0.96      0.96      3855



In [28]:
from sklearn import svm
classifier = svm.SVC(C=0.1,gamma=1,decision_function_shape='ovr')
classifier.fit(X_train,y_train)

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [29]:
y_pred=classifier.predict(X_test)

In [30]:
classifier.score(X_test,y_test)

0.7185473411154345

In [31]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.72      1.00      0.84      2770
        1.0       0.00      0.00      0.00      1085

avg / total       0.52      0.72      0.60      3855



f:\python35\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [32]:
#WEAK-AMPLIFICATION & RE-LABELLING
for x in range(0,len(minornoise)):
    temp=neigh.kneighbors([minornoise[x]])
    for xx in range(3):
        if major[temp[1][0][xx]] in majornoise:
            tt=major[temp[1][0][xx]]
            major.remove(tt)
            minor.append(tt)
            neigh.fit(major)

In [33]:
print(len(major))
print(len(minor))

9199
3650


In [34]:
y1=np.ones(len(minor))
y2=np.zeros(len(major))
yfinal=np.concatenate((y2,y1))
xfinal=np.concatenate((major,minor))

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( xfinal, yfinal, test_size=0.3, random_state=42)

In [36]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski' , p=2)
classifier.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [37]:
y_pred = classifier.predict(X_test)

In [38]:
classifier.score(X_test,y_test)

0.95486381322957203

In [39]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.94      1.00      0.97      2770
        1.0       0.99      0.85      0.91      1085

avg / total       0.96      0.95      0.95      3855



In [40]:
from sklearn.neural_network import MLPClassifier

In [41]:
model = MLPClassifier(hidden_layer_sizes=(100,100), max_iter=1000, batch_size=4,random_state=101)

In [42]:
model.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size=4, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=101,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [43]:
y_pred=model.predict(X_test)

In [44]:
model.score(X_test,y_test)

0.94422827496757455

In [45]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.99      0.93      0.96      2770
        1.0       0.85      0.98      0.91      1085

avg / total       0.95      0.94      0.95      3855



In [46]:
from sklearn import svm
classifier = svm.SVC(C=0.1,gamma=1,decision_function_shape='ovr')
classifier.fit(X_train,y_train)

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [47]:
y_pred=classifier.predict(X_test)

In [48]:
classifier.score(X_test,y_test)

0.7185473411154345

In [49]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.72      1.00      0.84      2770
        1.0       0.00      0.00      0.00      1085

avg / total       0.52      0.72      0.60      3855



f:\python35\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [60]:
#STRONG_AMPLIFICATION
for x in range(0,len(minorsafe)):
    temp=neigh.kneighbors([minorsafe[x]])
    for xx in range(3):
        if major[temp[1][0][xx]] in majorsafe:
            minor.append(minorsafe[x])

In [61]:
y1=np.ones(len(minor))
y2=np.zeros(len(major))
ynew=np.concatenate((y2,y1))
xnew=np.concatenate((major,minor))
model2 = KNeighborsClassifier(n_neighbors=5)
model2.fit(xnew,ynew)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [62]:
for x in range(0,len(minornoise)):
    if model2.predict([minornoise[x]]) == 1:
        temp=neigh.kneighbors([minorsafe[x]])
        for xx in range(0,len(temp[1][0])):
            if major[temp[1][0][xx]] in majorsafe:
                minor.append(minornoise[x])
    else:
        temp=neigh.kneighbors([minorsafe[x]],n_neighbors=5)
        for xx in range(0,len(temp[1][0])):
            if major[temp[1][0][xx]] in majorsafe:
                minor.append(minornoise[x])

In [63]:
y1=np.ones(len(minor))
y2=np.zeros(len(major))
yfinal=np.concatenate((y2,y1))
xfinal=np.concatenate((major,minor))

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( xfinal, yfinal, test_size=0.3, random_state=42)

In [82]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski' , p=2)
classifier.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [83]:
y_pred = classifier.predict(X_test)

In [84]:
classifier.score(X_test,y_test)

0.99168255068445677

In [68]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99      2736
        1.0       0.99      0.99      0.99      3035

avg / total       0.99      0.99      0.99      5771



In [69]:
from sklearn.neural_network import MLPClassifier

In [70]:
model = MLPClassifier(hidden_layer_sizes=(100,100), max_iter=1000, batch_size=4,random_state=101)

In [71]:
model.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size=4, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=101,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [72]:
y_pred=model.predict(X_test)

In [73]:
model.score(X_test,y_test)

0.96014555536302204

In [74]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.98      0.93      0.96      2736
        1.0       0.94      0.99      0.96      3035

avg / total       0.96      0.96      0.96      5771



In [75]:
from sklearn import svm
classifier = svm.SVC(C=0.1,gamma=1,decision_function_shape='ovr')
classifier.fit(X_train,y_train)

SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [76]:
y_pred=classifier.predict(X_test)

In [77]:
classifier.score(X_test,y_test)

0.52590538901403572

In [78]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00      2736
        1.0       0.53      1.00      0.69      3035

avg / total       0.28      0.53      0.36      5771



f:\python35\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
